In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "5"
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import Levenshtein
import cv2
from PIL import Image
from matplotlib import pyplot as plt
import seaborn as sns
import time
import torch.backends.cudnn as cudnn
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
from torch.nn.utils.rnn import pack_padded_sequence
import torchvision
import PIL.Image as Image
from torchvision.transforms import ToTensor, ToPILImage
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch
from torch import nn
from nltk.translate.bleu_score import corpus_bleu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
 # Increase Batch size based on your hardware capacity, if you have powerful GPU try BS=128,256...
train_path=r'../input/bird-species-classification-220-categories/Train'
test_path=r'../input/bird-species-classification-220-categories/Test'

In [2]:
caption_class = {}
images_class = []
with os.scandir('../input/captions/text/') as entries:
    for entry in entries:
        
       # with os.scandir(entry) as entries:
       if os.path.isdir(entry):
            caption_class[entry.name.split('.')[1]] = entry.name
        #caption_class.append(entry.name.split('.')[1])
with os.scandir('../input/bird-species-classification-220-categories/Train') as entrie:
    for entry in entrie:
        images_class.append(entry.name)

NameError: name 'os' is not defined

In [ ]:
print(len(images_class))

<h3>Put the image paths and caption paths into lists to facilitate their usage during the loading and processing of the dataset, especially for handling both train and test sets.</h3>

In [ ]:
import cv2
path_caption = '../input/captions/text/'

train_path=r'../input/bird-species-classification-220-categories/Train'
test_path=r'../input/bird-species-classification-220-categories/Test'
images_paths = []
captions_paths = []
images_test = []
captions_test = []
name_captin = []
test_cap = []
#embedding_paths = []
for c_image in images_class:
    for image_name in os.listdir(os.path.join(train_path,c_image)):
        d=os.path.join(train_path,c_image)
        image_path =os.path.join(d,image_name)
        images_paths.append(image_path)
        name_captin_file = image_name[:-4]+".txt"
        d=os.path.join(path_caption,caption_class[c_image])
        caption_path =os.path.join(d,name_captin_file)
        captions_paths.append(caption_path)
for c_image in images_class:
    for image_name in os.listdir(os.path.join(test_path,c_image)):
        d=os.path.join(test_path,c_image)
        image_path =os.path.join(d,image_name)
        images_test.append(image_path)
        #img = cv2.imread(image_path)
        name_captin_file = image_name[:-4]+".txt"
        d=os.path.join(path_caption,caption_class[c_image])
        caption_path =os.path.join(d,name_captin_file)
       # embedding_path = os.path.join(d,embedding_caption)
        captions_test.append(caption_path)
    

In [ ]:
print(len(images_paths),len(captions_paths))
print(len(images_test),len(captions_test))

<H2>Upload the pre-trained BERT and his tokenizer</H2>

In [ ]:
from transformers import AutoTokenizer, AutoModel

# Mini:   asafaya/bert-mini-arabic
# Medium: asafaya/bert-medium-arabic
# Base:   asafaya/bert-base-arabic
# Large:  asafaya/bert-large-arabic

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")


<H2> Extract all the tokens of the dataset, including train and test sets </H2>

In [ ]:
toknaz = []
s = captions_paths + captions_test
for i in s:
    with open(i,'r') as file:
        lines = file.read()
        m = tokenizer.encode(lines)
        for s in m:
            if s not in toknaz:
                toknaz.append(s)
print(len(toknaz)) 

In [3]:
model.to(device)

NameError: name 'model' is not defined

<H2> "Determine the maximum text length and use it as the sequences length." </H2>

In [ ]:
s = captions_paths + captions_test
max_seq = 0
for i in s:
    with open(i,'r') as file:
        lines = file.read()
        m = len(tokenizer.encode(lines))
        if max_seq < m:
            max_seq = m

In [ ]:
max_seq

<H2> Clean the text, and find the maxumim langth of the text </H2>

In [ ]:
from nltk.tokenize import RegexpTokenizer
tokens_new = []
s = captions_paths + captions_test
max_ = 0
for i in s:
    lis = []
    with open(i, "r") as f:
        captions = f.read().split('\n')
    for cap in captions:
        cap = cap.replace("\ufffd\ufffd", " ")
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(cap.lower())
        
        for t in tokens:
            lis.append(t)
            t = t.encode('ascii', 'ignore').decode('ascii')
            if t not in tokens_new:
                tokens_new.append(t)
    if max_ < len(lis):
        max_ = len(lis)
        
print(len(tokens_new))
print(max_)

<H2> Add the end token</H2>

In [ ]:
ixtoword = {}
ixtoword[0] = '<end>'
wordtoix = {}
wordtoix['<end>'] = 0
ix = 1
for w in tokens_new:
    wordtoix[w] = ix
    ixtoword[ix] = w
    ix += 1

<H2> The pre-proccesssing of iamges and thier captions </H2>

In [4]:
def read_text(path):
    tokens_new = []
    with open(path, "r") as f:
        captions = f.read().split('\n')
    for cap in captions:
        cap = cap.replace("\ufffd\ufffd", " ")
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(cap.lower())
        
        for t in tokens:
            lis.append(t)
            t = t.encode('ascii', 'ignore').decode('ascii')
            if t not in tokens_new:
                tokens_new.append(t)
    return tokens_new

def text_to_tensor1(path,max_len):
    seq_of_words = read_text(path)
    m = [wordtoix[w] for w in tokens_new]
    len_seq = len(m)
    tensor = torch.LongTensor(F.pad(torch.LongTensor(m), pad=(0, (max_len+1 )- len_seq) , mode='constant', value=0)) 
    return tensor,len_seq



def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
        
train_transforms =  transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5]),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

test_transforms =  transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

class dataset1(torch.utils.data.Dataset):

    def __init__(self,images_paths,captions_paths,max_len,transform=None):
        self.images_paths = images_paths
        self.captions_paths = captions_paths
        self.max_len = max_len 
        self.transform = transform 
         
    #dataset length
    def __len__(self):
        self.filelength = len(self.images_paths)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
     #   img_path = self.file_list[idx]
        img = pil_loader(self.images_paths[idx].strip())
        tensor_caption,len_seq = text_to_tensor1(self.captions_paths[idx],self.max_len)
        #seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long(
        img_transformed = self.transform(img)   
        #img_transformed = img_transformed.expand(3,256,256)
        return img_transformed,tensor_caption,len_seq

NameError: name 'transforms' is not defined

In [ ]:
def text_to_tensor(path,max_len):
    with open(path,'r') as file:
        lines = file.read()
        m = tokenizer.encode(lines)
        m.append(0)
        len_seq = len(m)
        tensor = torch.LongTensor(F.pad(torch.LongTensor(m), pad=(0, (max_len+1 )- len_seq) , mode='constant', value=0)) 
    return tensor,len_seq



def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
        
train_transforms =  transforms.Compose([
        transforms.Resize(256),
        transforms.RandomCrop(256),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        #transforms.Normalize([0.5], [0.5]),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

test_transforms =  transforms.Compose([
        transforms.Resize((256,256)),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

class dataset(torch.utils.data.Dataset):

    def __init__(self,images_paths,captions_paths,max_len,transform=None):
        self.images_paths = images_paths
        self.captions_paths = captions_paths
        self.max_len = max_len 
        self.transform = transform 
         
    #dataset length
    def __len__(self):
        self.filelength = len(self.images_paths)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
     #   img_path = self.file_list[idx]
        img = pil_loader(self.images_paths[idx].strip())
        tensor_caption,len_seq = text_to_tensor(self.captions_paths[idx],self.max_len)
        #seq_tensor = Variable(torch.zeros((len(vectorized_seqs), seq_lengths.max()))).long(
        img_transformed = self.transform(img)   
        #img_transformed = img_transformed.expand(3,256,256)
        return img_transformed,tensor_caption,len_seq

In [ ]:
print(len(images_paths),len(captions_paths))
print(len(images_test),len(captions_test))

In [ ]:
max_seq=300
images_paths  = images_paths[:9408]
captions_paths = captions_paths[:9408]
test_images = images_test[:2368] 
test_captions = captions_test[:2368] 
train_data = dataset(images_paths,captions_paths,max_seq,transform=train_transforms)
train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size=32, shuffle=True)
test_data = dataset(test_images,test_captions,max_seq,transform=test_transforms)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size=32, shuffle=False)


In [ ]:
print(len(train_data))
print(len(train_loader))
print(len(test_data))
print(len(test_loader))

<H2> Define the GAN generator and discriminator</H2>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F
from collections import OrderedDict



class NetG(nn.Module):
    def __init__(self, ngf=64, nz=100):
        super(NetG, self).__init__()
        self.ngf = ngf
        self.cr = nn.Linear(768,256)
        self.fc = nn.Linear(nz, ngf*8*4*4)
        self.block0 = G_Block(ngf * 8, ngf * 8)#4x4
        self.block1 = G_Block(ngf * 8, ngf * 8)#4x4
        self.block2 = G_Block(ngf * 8, ngf * 8)#8x8
        self.block3 = G_Block(ngf * 8, ngf * 8)#16x16
        self.block4 = G_Block(ngf * 8, ngf * 4)#32x32
        self.block5 = G_Block(ngf * 4, ngf * 2)#64x64
        self.block6 = G_Block(ngf * 2, ngf * 1)#128x128

        self.conv_img = nn.Sequential(
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ngf, 3, 3, 1, 1),
            nn.Tanh(),
        )

    def forward(self, x, c):

        out = self.fc(x)
        out = out.view(x.size(0), 8*self.ngf, 4, 4)
        c = self.cr(c)
        out = self.block0(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block1(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block2(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block3(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block4(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block5(out,c)

        out = F.interpolate(out, scale_factor=2)
        out = self.block6(out,c)

        out = self.conv_img(out)

        return out



class G_Block(nn.Module):

    def __init__(self, in_ch, out_ch):
        super(G_Block, self).__init__()

        self.learnable_sc = in_ch != out_ch 
        self.c1 = nn.Conv2d(in_ch, out_ch, 3, 1, 1)
        self.c2 = nn.Conv2d(out_ch, out_ch, 3, 1, 1)
        self.affine0 = affine(in_ch)
        self.affine1 = affine(in_ch)
        self.affine2 = affine(out_ch)
        self.affine3 = affine(out_ch)
        self.gamma = nn.Parameter(torch.zeros(1))
        if self.learnable_sc:
            self.c_sc = nn.Conv2d(in_ch,out_ch, 1, stride=1, padding=0)

    def forward(self, x, y=None):
        return self.shortcut(x) + self.gamma * self.residual(x, y)

    def shortcut(self, x):
        if self.learnable_sc:
            x = self.c_sc(x)
        return x

    def residual(self, x, y=None):
        h = self.affine0(x, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        h = self.affine1(h, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        h = self.c1(h)
        
        h = self.affine2(h, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        h = self.affine3(h, y)
        h = nn.LeakyReLU(0.2,inplace=True)(h)
        return self.c2(h)



class affine(nn.Module):

    def __init__(self, num_features):
        super(affine, self).__init__()

        self.fc_gamma = nn.Sequential(OrderedDict([
            ('linear1',nn.Linear(256, 256)),
            ('relu1',nn.ReLU(inplace=True)),
            ('linear2',nn.Linear(256, num_features)),
            ]))
        self.fc_beta = nn.Sequential(OrderedDict([
            ('linear1',nn.Linear(256, 256)),
            ('relu1',nn.ReLU(inplace=True)),
            ('linear2',nn.Linear(256, num_features)),
            ]))
        self._initialize()

    def _initialize(self):
        nn.init.zeros_(self.fc_gamma.linear2.weight.data)
        nn.init.ones_(self.fc_gamma.linear2.bias.data)
        nn.init.zeros_(self.fc_beta.linear2.weight.data)
        nn.init.zeros_(self.fc_beta.linear2.bias.data)

    def forward(self, x, y=None):

        weight = self.fc_gamma(y)
        bias = self.fc_beta(y)        

        if weight.dim() == 1:
            weight = weight.unsqueeze(0)
        if bias.dim() == 1:
            bias = bias.unsqueeze(0)

        size = x.size()
        weight = weight.unsqueeze(-1).unsqueeze(-1).expand(size)
        bias = bias.unsqueeze(-1).unsqueeze(-1).expand(size)
        return weight * x + bias


class D_GET_LOGITS(nn.Module):
    def __init__(self, ndf):
        super(D_GET_LOGITS, self).__init__()
        self.df_dim = ndf
        self.kl = nn.Linear(768,256)
        self.joint_conv = nn.Sequential(
            nn.Conv2d(ndf * 16+256, ndf * 2, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.2,inplace=True),
            nn.Conv2d(ndf * 2, 1, 4, 1, 0, bias=False),
        )

    def forward(self, out, y):
        y = self.kl(y)
        y = y.view(-1, 256, 1, 1)
        y = y.repeat(1, 1, 4, 4)
        h_c_code = torch.cat((out, y), 1)
        out = self.joint_conv(h_c_code)
        return out





class NetD(nn.Module):
    def __init__(self, ndf):
        super(NetD, self).__init__()
        self.conv_img = nn.Conv2d(3, ndf, 3, 1, 1)#128
        self.block0 = resD(ndf * 1, ndf * 2)#64
        self.block1 = resD(ndf * 2, ndf * 4)#32
        self.block2 = resD(ndf * 4, ndf * 8)#16
        self.block3 = resD(ndf * 8, ndf * 16)#8
        self.block4 = resD(ndf * 16, ndf * 16)#4
        self.block5 = resD(ndf * 16, ndf * 16)#4

        self.COND_DNET = D_GET_LOGITS(ndf)

    def forward(self,x):

        out = self.conv_img(x)
        out = self.block0(out)
        out = self.block1(out)
        out = self.block2(out)
        out = self.block3(out)
        out = self.block4(out)
        out = self.block5(out)

        return out




class resD(nn.Module):
    def __init__(self, fin, fout, downsample=True):
        super().__init__()
        self.downsample = downsample
        self.learned_shortcut = (fin != fout)
        self.conv_r = nn.Sequential(
            nn.Conv2d(fin, fout, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            nn.Conv2d(fout, fout, 3, 1, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.conv_s = nn.Conv2d(fin,fout, 1, stride=1, padding=0)
        self.gamma = nn.Parameter(torch.zeros(1))

    def forward(self, x, c=None):
        return self.shortcut(x)+self.gamma*self.residual(x)

    def shortcut(self, x):
        if self.learned_shortcut:
            x = self.conv_s(x)
        if self.downsample:
            return F.avg_pool2d(x, 2)
        return x

    def residual(self, x):
        return self.conv_r(x)

In [ ]:
### device = "cuda"
import torchvision.utils as vutils
import random
batch_size = 32
random.seed(100)
np.random.seed(100)
torch.manual_seed(100)
torch.cuda.manual_seed_all(100)
CUDA_LAUNCH_BLOCKING=1
netG = NetG(batch_size, 100).to(device)
netD = NetD(batch_size).to(device)
optimizerG = torch.optim.Adam(netG.parameters(), lr=0.0001, betas=(0.0, 0.9))
optimizerD = torch.optim.Adam(netD.parameters(), lr=0.0004, betas=(0.0, 0.9))
checkpoint_G = torch.load('../input/modul-bert-490/netG_vBE490.pt')
netG.load_state_dict(checkpoint_G['model_state_dict'])
optimizerG.load_state_dict(checkpoint_G['optimizer_state_dict'])
epoch_ = checkpoint_G['epoch']

checkpoint_D = torch.load('../input/modul-bert-490/netD_vBE490.pt')
netD.load_state_dict(checkpoint_D['model_state_dict'])
optimizerD.load_state_dict(checkpoint_D['optimizer_state_dict'])
netG.train()
netD.train()


<H2> The Training loop of the GAN</H2>

In [ ]:
#from miscc.utils import mkdir_p
#from miscc.config import cfg, cfg_from_file

for epoch in range(epoch_ +1,epoch_ +41):
    i = 0
    for images,captions,cap_lens in train_loader:
        cap_lens,index = torch.sort(cap_lens,descending = True)
        captions = captions[index]
        images = images[index]
        #print(images.size())
        with torch.no_grad():
            last_hidden_states = model(captions.to(device))
            sent_emb = last_hidden_states[0][:,-1,:].detach().to(device)
        noise = torch.randn(batch_size, 100)
        noise=noise.to(device)
        fake = netG(noise,sent_emb.to(device))  
            #imgs=images[0].to(device)
        real_features = netD(images.to(device))
        output = netD.COND_DNET(real_features,sent_emb.to(device))
        errD_real = torch.nn.ReLU()(1.0 - output).mean()
        output = netD.COND_DNET(real_features[:(batch_size - 1)].to(device), sent_emb[1:batch_size].to(device))
        errD_mismatch = torch.nn.ReLU()(1.0 + output).mean()
            #print("errD_mismatch :",errD_mismatch.size())
            # synthesize fake images
        fake_features = netD(fake.detach()) 
        errD_fake = netD.COND_DNET(fake_features.to(device),sent_emb.to(device))
        errD_fake = torch.nn.ReLU()(1.0 + errD_fake).mean()          
        errD = errD_real + (errD_fake + errD_mismatch)/2.0
            #print("errD : ",errD)
        optimizerD.zero_grad()
        optimizerG.zero_grad()
        errD.backward()
        optimizerD.step()
            #MA-GP
        interpolated = (images.data).requires_grad_()
        sent_inter = (sent_emb.data).requires_grad_()
        features = netD(interpolated.to(device))
        out = netD.COND_DNET(features,sent_inter)
        grads = torch.autograd.grad(outputs=out,
                                    inputs=(interpolated,sent_inter),
                                    grad_outputs=torch.ones(out.size()).to(device),
                                    retain_graph=True,
                                    create_graph=True,
                                    only_inputs=True)
        grad0 = grads[0].view(grads[0].size(0), -1).to(device)
        grad1 = grads[1].view(grads[1].size(0), -1).to(device)
        grad = torch.cat((grad0,grad1),dim=1)                        
        grad_l2norm = torch.sqrt(torch.sum(grad ** 2, dim=1))
        d_loss_gp = torch.mean((grad_l2norm) ** 6)
        d_loss = 2.0 * d_loss_gp
        optimizerD.zero_grad()
        optimizerG.zero_grad()
        d_loss.backward()
        optimizerD.step()
        i += 1  
            # update G
        features = netD(fake)
        output = netD.COND_DNET(features,sent_emb)
        errG = - output.mean()
        optimizerG.zero_grad()
        optimizerD.zero_grad()
        errG.backward()
        optimizerG.step()
        print('[%d/%d][%d/%d] Loss_D: %.3f Loss_G %.3f'
                % (epoch, 600, i, len(train_loader), errD.item(), errG.item()))
        if (epoch+1) % 2 == 0 and i == 294:
            print("save ................")
            vutils.save_image(fake.data,
                        '%s/fake_samples_epoch_%03d.png' % ('./', epoch+1),
                        normalize=True)
            torch.save({
            'model_state_dict': netD.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            },'./netD_vBE520.pt')
            
            torch.save({
            'epoch': epoch,
            'model_state_dict': netG.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            },'./netG_vBE520.pt')


In [ ]:
os.mkdir('./Output/')

<H2> The evaluation of the model, the model take the text and generate the images</H2>

In [ ]:
#os.mkdir('./Output/')
batch_size = 32
save_dir = './Output/'
d = 0
t = 0
for i in range(1):  # (cfg.TEXT.CAPTIONS_PER_IMAGE):
    for images,captions,cap_lens in test_loader:
        cap_lens,index = torch.sort(cap_lens,descending = True)
        captions = captions[index]
        images = images[index]
        with torch.no_grad():
            last_hidden_states = model(captions.to(device))
            sent_emb = last_hidden_states[0][:,-1,:].detach().to(device)
            #sent_emb = sent_emb.transpose(0, 1).contiguous()
            
        #print("captions :",captions.size())
        #print("captions :",captions[3])
        #print("images :",images.size())
        #words_embs, sent_emb = text_encoder(captions.to(device),cap_lens.to(device),hidden)
        #words_embs, sent_emb = words_embs.detach().to(device), sent_emb.detach().to(device)
            #######################################################
            # (2) Generate fake images
            ######################################################
        with torch.no_grad():
            noise = torch.randn(batch_size, 100)
            noise=noise.to(device)
            fake_imgs = netG(noise,sent_emb)
        print(">>>>>>>>batch : ",d)
        d += 1
        for j in range(batch_size):
            s_tmp = './Output/'
            im = fake_imgs[j].data.cpu().numpy()
                # [-1, 1] --> [0, 255]
            arab_cap = captions[j]
            im = (im + 1.0) * 127.5
            im = im.astype(np.uint8)
            im = np.transpose(im, (1, 2, 0))
            im = Image.fromarray(im)
            arab_cap = tokenizer.decode(arab_cap)
            cap_path = '%s_%3d.txt' % (s_tmp,t)
            fullpath = '%s_%3d.png' % (s_tmp,t)
            t += 1
            im.save(fullpath)
            #with open(cap_path,'w') as f:
             #   f.write(arab_cap)
                

In [ ]:
!zip -r Output_bert.zip ./

<H2> The rest of code is about the evaluation using FID and IS</H2>

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models


class InceptionV3(nn.Module):
    """Pretrained InceptionV3 network returning feature maps"""

    # Index of default block of inception to return,
    # corresponds to output of final average pooling
    DEFAULT_BLOCK_INDEX = 3

    # Maps feature dimensionality to their output blocks indices
    BLOCK_INDEX_BY_DIM = {
        64: 0,   # First max pooling features
        192: 1,  # Second max pooling featurs
        768: 2,  # Pre-aux classifier features
        2048: 3  # Final average pooling features
    }

    def __init__(self,
                 output_blocks=[DEFAULT_BLOCK_INDEX],
                 resize_input=True,
                 normalize_input=True,
                 requires_grad=False):
        """Build pretrained InceptionV3
        Parameters
        ----------
        output_blocks : list of int
            Indices of blocks to return features of. Possible values are:
                - 0: corresponds to output of first max pooling
                - 1: corresponds to output of second max pooling
                - 2: corresponds to output which is fed to aux classifier
                - 3: corresponds to output of final average pooling
        resize_input : bool
            If true, bilinearly resizes input to width and height 299 before
            feeding input to model. As the network without fully connected
            layers is fully convolutional, it should be able to handle inputs
            of arbitrary size, so resizing might not be strictly needed
        normalize_input : bool
            If true, normalizes the input to the statistics the pretrained
            Inception network expects
        requires_grad : bool
            If true, parameters of the model require gradient. Possibly useful
            for finetuning the network
        """
        super(InceptionV3, self).__init__()

        self.resize_input = resize_input
        self.normalize_input = normalize_input
        self.output_blocks = sorted(output_blocks)
        self.last_needed_block = max(output_blocks)

        assert self.last_needed_block <= 3, \
            'Last possible output block index is 3'

        self.blocks = nn.ModuleList()

        inception = models.inception_v3(pretrained=True)

        # Block 0: input to maxpool1
        block0 = [
            inception.Conv2d_1a_3x3,
            inception.Conv2d_2a_3x3,
            inception.Conv2d_2b_3x3,
            nn.MaxPool2d(kernel_size=3, stride=2)
        ]
        self.blocks.append(nn.Sequential(*block0))

        # Block 1: maxpool1 to maxpool2
        if self.last_needed_block >= 1:
            block1 = [
                inception.Conv2d_3b_1x1,
                inception.Conv2d_4a_3x3,
                nn.MaxPool2d(kernel_size=3, stride=2)
            ]
            self.blocks.append(nn.Sequential(*block1))

        # Block 2: maxpool2 to aux classifier
        if self.last_needed_block >= 2:
            block2 = [
                inception.Mixed_5b,
                inception.Mixed_5c,
                inception.Mixed_5d,
                inception.Mixed_6a,
                inception.Mixed_6b,
                inception.Mixed_6c,
                inception.Mixed_6d,
                inception.Mixed_6e,
            ]
            self.blocks.append(nn.Sequential(*block2))

        # Block 3: aux classifier to final avgpool
        if self.last_needed_block >= 3:
            block3 = [
                inception.Mixed_7a,
                inception.Mixed_7b,
                inception.Mixed_7c,
                nn.AdaptiveAvgPool2d(output_size=(1, 1))
            ]
            self.blocks.append(nn.Sequential(*block3))

        for param in self.parameters():
            param.requires_grad = requires_grad

    def forward(self, inp):
        """Get Inception feature maps
        Parameters
        ----------
        inp : torch.autograd.Variable
            Input tensor of shape Bx3xHxW. Values are expected to be in 
            range (0, 1)
        Returns
        -------
        List of torch.autograd.Variable, corresponding to the selected output 
        block, sorted ascending by index
        """
        outp = []
        x = inp

        if self.resize_input:
            x = F.upsample(x, size=(299, 299), mode='bilinear', align_corners=True)

        if self.normalize_input:
            x = x.clone()
            x[:, 0] = x[:, 0] * (0.229 / 0.5) + (0.485 - 0.5) / 0.5
            x[:, 1] = x[:, 1] * (0.224 / 0.5) + (0.456 - 0.5) / 0.5
            x[:, 2] = x[:, 2] * (0.225 / 0.5) + (0.406 - 0.5) / 0.5

        for idx, block in enumerate(self.blocks):
            x = block(x)
            if idx in self.output_blocks:
                outp.append(x)

            if idx == self.last_needed_block:
                break

        return outp

In [7]:
block_idx = InceptionV3.BLOCK_INDEX_BY_DIM[2048]
model_incp = InceptionV3([block_idx])
model_incp=model_incp.to(device)

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth


  0%|          | 0.00/104M [00:00<?, ?B/s]

NameError: name 'device' is not defined

In [ ]:
def calculate_activation_statistics(images,model,batch_size=128, dims=2048,
                    cuda=False):
    model.eval()
    act=np.empty((len(images), dims))
    
    if cuda:
        batch=images.cuda()
    else:
        batch=images
    pred = model(batch)[0]

        # If model output is not scalar, apply global spatial average pooling.
        # This happens if you choose a dimensionality not equal 2048.
    if pred.size(2) != 1 or pred.size(3) != 1:
        pred = adaptive_avg_pool2d(pred, output_size=(1, 1))

    act= pred.cpu().data.numpy().reshape(pred.size(0), -1)
    
    mu = np.mean(act, axis=0)
    sigma = np.cov(act, rowvar=False)
    return mu, sigma

In [ ]:
def calculate_frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    """Numpy implementation of the Frechet Distance.
    The Frechet distance between two multivariate Gaussians X_1 ~ N(mu_1, C_1)
    and X_2 ~ N(mu_2, C_2) is
            d^2 = ||mu_1 - mu_2||^2 + Tr(C_1 + C_2 - 2*sqrt(C_1*C_2)).
    """

    mu1 = np.atleast_1d(mu1)
    mu2 = np.atleast_1d(mu2)

    sigma1 = np.atleast_2d(sigma1)
    sigma2 = np.atleast_2d(sigma2)

    assert mu1.shape == mu2.shape, \
        'Training and test mean vectors have different lengths'
    assert sigma1.shape == sigma2.shape, \
        'Training and test covariances have different dimensions'

    diff = mu1 - mu2

    
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        msg = ('fid calculation produces singular product; '
               'adding %s to diagonal of cov estimates') % eps
        print(msg)
        offset = np.eye(sigma1.shape[0]) * eps
        covmean = linalg.sqrtm((sigma1 + offset).dot(sigma2 + offset))

    
    if np.iscomplexobj(covmean):
        if not np.allclose(np.diagonal(covmean).imag, 0, atol=1e-3):
            m = np.max(np.abs(covmean.imag))
            raise ValueError('Imaginary component {}'.format(m))
        covmean = covmean.real

    tr_covmean = np.trace(covmean)

    return (diff.dot(diff) + np.trace(sigma1) +
            np.trace(sigma2) - 2 * tr_covmean)

In [ ]:
def calculate_fretchet(images_real,images_fake,model):
    mu_1,std_1=calculate_activation_statistics(images_real,model,cuda=True)
    mu_2,std_2=calculate_activation_statistics(images_fake,model,cuda=True)
    
    #"""get fretched distance"""
    fid_value = calculate_frechet_distance(mu_1, std_1, mu_2, std_2)
    return fid_value

In [ ]:
def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = Image.open(f)
        return img.convert('RGB')
    
        
train_transforms =  transforms.Compose([
        transforms.Resize(299),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                             ])

class dataset(torch.utils.data.Dataset):

    def __init__(self,images_paths,transform=None):
        self.images_paths = images_paths 
        self.transform = transform 
         
    #dataset length
    def __len__(self):
        self.filelength = len(self.images_paths)
        return self.filelength
    
    #load an one of images
    def __getitem__(self,idx):
     #   img_path = self.file_list[idx]
        img = pil_loader(self.images_paths[idx].strip())
        img_transformed = self.transform(img)   
        #img_transformed = img_transformed.expand(3,256,256)
        return img_transformed

In [ ]:
path = "./Output/"
output_images = []
with os.scandir('./Output') as entrie:
    for entry in entrie:
        output_images.append(path+entry.name)

In [ ]:
generated_images = dataset(output_images,transform=train_transforms)

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.nn import functional as F
import torch.utils.data

from torchvision.models.inception import inception_v3

import numpy as np
from scipy.stats import entropy

def inception_score(imgs, cuda=True, batch_size=32, resize=True, splits=1):
    """Computes the inception score of the generated images imgs
    imgs -- Torch dataset of (3xHxW) numpy images normalized in the range [-1, 1]
    cuda -- whether or not to run on GPU
    batch_size -- batch size for feeding into Inception v3
    splits -- number of splits
    """
    N = len(imgs)

    assert batch_size > 0
    assert N > batch_size

    # Set up dtype
    if cuda:
        dtype = torch.cuda.FloatTensor
    else:
        if torch.cuda.is_available():
            print("WARNING: You have a CUDA device, so you should probably set cuda=True")
        dtype = torch.FloatTensor

    # Set up dataloader
    dataloader = torch.utils.data.DataLoader(imgs, batch_size=batch_size)

    # Load inception model
    inception_model = inception_v3(pretrained=True, transform_input=False).type(dtype)
    inception_model.eval();
    up = nn.Upsample(size=(299, 299), mode='bilinear').type(dtype)
    def get_pred(x):
        if resize:
            x = up(x)
        x = inception_model(x)
        return F.softmax(x).data.cpu().numpy()

    # Get predictions
    preds = np.zeros((N, 1000))

    for i, batch in enumerate(dataloader, 0):
        batch = batch.type(dtype)
        batchv = Variable(batch)
        batch_size_i = batch.size()[0]

        preds[i*batch_size:i*batch_size + batch_size_i] = get_pred(batchv)

    # Now compute the mean kl-div
    split_scores = []

    for k in range(splits):
        part = preds[k * (N // splits): (k+1) * (N // splits), :]
        py = np.mean(part, axis=0)
        scores = []
        for i in range(part.shape[0]):
            pyx = part[i, :]
            scores.append(entropy(pyx, py))
        split_scores.append(np.exp(np.mean(scores)))

    return np.mean(split_scores), np.std(split_scores)


import torchvision.datasets as dset
import torchvision.transforms as transforms

print ("Calculating Inception Score...")
print (inception_score(generated_images, cuda=True, batch_size=32, resize=True, splits=10))

In [ ]:
from scipy import linalg
from PIL import Image
batch_size = 32
save_dir = './Output/'
frech = []
d = 0
t = 0
    
for i in range(1):  # (cfg.TEXT.CAPTIONS_PER_IMAGE):
    for images,captions,cap_lens in test_loader:
        #cap_lens,index = torch.sort(cap_lens,descending = True)
        #captions = captions[index]
        #images = images[index]
        with torch.no_grad():
            #last_hidden_states = model(captions.to(device))
            #sent_emb = last_hidden_states[0][:,-1,:].detach().to(device)
            #sent_emb = sent_emb.transpose(0, 1).contiguous()
            
            hidden = text_encoder.init_hidden(batch_size)
            # words_embs: batch_size x nef x seq_len
            # sent_emb: batch_size x nef
            words_embs, sent_emb = text_encoder(captions.to(device), cap_lens.to(device), hidden)
            words_embs, sent_emb = words_embs.detach(), sent_emb.detach()
            #######################################################
            # (2) Generate fake images
            ######################################################
        with torch.no_grad():
            noise = torch.randn(batch_size, 100)
            noise=noise.to(device)
            fake_imgs = netG(noise,sent_emb)
        fd = calculate_fretchet(images,fake_imgs,model_incp)
        print(fd)
        frech.append(fd)
        
        